In [1]:
import pickle as pkl
import numpy as np
import pandas as pd
import operator
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.model_selection import cross_val_predict, cross_val_score, KFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Imputer, StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.metrics import log_loss, roc_auc_score, f1_score
from xgboost import XGBClassifier

/home/joe/kaggle/instacart/.env/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
X = pd.read_csv("../data/reorder_features_test.csv", index_col=0,
                usecols=["order_id", "product_id"])
X.head()

FileNotFoundError: File b'../data/reorder_features_test.csv' does not exist

In [12]:
reorders = pd.read_csv("../data/reorder_preds_train.csv", index_col=0)
none_preds = pd.read_csv("../data/none_preds_train.csv", index_col=0)

/home/joe/kaggle/instacart/.env/lib/python3.5/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [21]:
reorders.index = X["order_id"]
reorders["product_id"] = X["product_id"].values

In [23]:
reorders.head()

,xgb,linear,mean,product_id
order_id,,,,
1187899,0.598291,0.820084,0.709188,196
1187899,0.598291,0.800088,0.699189,12427
1187899,0.598291,0.763885,0.681088,10258
1187899,0.598291,0.723740,0.661016,25133
1187899,0.341717,0.481698,0.411708,46149


In [44]:
truth = pd.read_csv('../data/order_products__train.csv', dtype=np.int32, index_col=0)
truth = truth.groupby("order_id")[["product_id", "reordered"]].apply(lambda x: list(x.loc[x["reordered"] == True, "product_id"]))
truth.shape

/home/joe/kaggle/instacart/.env/lib/python3.5/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(131209,)

In [ ]:
for thr in np.linspace(0.1, 0.2, 3):
    for thr2 in np.linspace(0.1, 0.5, 3):
        preds = reorders[reorders["mean"] >= thr].groupby(level=0)["product_id"].apply(list).loc[truth.index]
        nulls = preds[(preds.isnull())|(none_preds["mean"] > thr2)].index
        for i in nulls:
            preds.loc[i] = [-1]
        score = f1_score(truth.values, preds.values)
        print(thr, thr2, score)

0.1 0.1 0.318160960413
0.1 0.3 0.335695359146
